##### [ 가변 길이의 텍스트 처리를 위한 모델 ]
- 임베딩층 : nn.EmbeddingBag 
- 구현
    * 입력 텍스트의 길이를 측정해서 길이만큼 추출 후 진행

[1] 모듈 로딩 및 데이터 준비<hr>

In [1]:
## 모듈 로딩
import torch 
import torch.nn as nn 
from torchtext.datasets import AG_NEWS

c:\Users\KDP-35\anaconda3\envs\NLP\lib\site-packages\torchtext\datasets\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-35\anaconda3\envs\NLP\lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
## 데이터 준비 
SAVE_DIR = '../data/'

[2] 데이터 로딩 및 확인<hr>

In [3]:
## pytorch의 torchtext의 내장 데이터셋 
trainDS, testDS = AG_NEWS()

c:\Users\KDP-35\anaconda3\envs\NLP\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


 [3] 데이터로더 생성 <hr>

In [4]:
## 관련 모듈 로딩
from torch.utils.data import DataLoader                 ## 데이터셋, 데이터로더 관련 모듈

## 토커나이저, 단어사전 관련 모듈
from torchtext.data.utils import get_tokenizer          ## 토커나이저 인스턴스 추출
from torchtext.vocab import build_vocab_from_iterator   ## 데이터셋에서 단어사전 생성 함수
from nltk.corpus import stopwords                       ## 불용어 데이터셋


c:\Users\KDP-35\anaconda3\envs\NLP\lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\KDP-35\anaconda3\envs\NLP\lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


[3-1] 단어사전 생성 <hr>

In [5]:
### ==> 특별 문자 토큰
UNK, PAD  = '<UNK>',  '<PAD>'
STOPWORDS = stopwords.words('english')
print(f'STOPWORDS : {STOPWORDS[:10]}')

### ==> 토커나이즈 생성
tokenizer = get_tokenizer('basic_english')

STOPWORDS : ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']


In [6]:
### ===> 토큰 제너레이터 함수 : 데이터 추출하여 토큰화 
def yield_tokens(data_iter):
    for label, news in data_iter:
        # 라벨, 뉴스 --> 텍스트 토큰화
        tokens = tokenizer(news)
        # print(f'[tokens 1] => {len(tokens)}')
        # tokens =[token for token in tokens if token not in STOPWORDS]
        # print(f'[tokens 2] => {len(tokens)}')
        yield tokens

In [7]:
### ===> 단어사전 생성
vocab = build_vocab_from_iterator(yield_tokens(trainDS), 
                                  specials=[PAD, UNK])

### ===> <PAD> 인덱스 0으로 설정
vocab.set_default_index(vocab[PAD])

print(f'[vocab] {len(vocab)}개')

[vocab] 95812개


In [8]:
### ===> 텍스트 >>>> 정수 인코딩
text_pipeline = lambda x: vocab(tokenizer(x))

### ===> 레이틀 >>> 정수 인코딩 (0~3) : 1~4
label_pipeline = lambda x: int(x) - 1

[3-2] 단어사전 생성 <hr>

In [9]:
## 배치크기만큼 데이터 로딩 시 위치 지정 위한 설정 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
## ----------------------------------------------------------------------
## 함수기능 : 배치크기 만큼 데이터셋 로딩해서 토큰 + 텐서화 진행 후 반환
## ----------------------------------------------------------------------
def collate_batch(batch):
    ## 라벨, 뉴스, 뉴스기사 시작 위치값 저장 변수
    label_list, news_list, offsets = [], [], [0]

    ## 1개씩 라벨과 뉴스 기사 추출
    for label, news in batch:
        ## 라벨 인코딩 후 추가 : 1 ~ 4 => 0 ~ 3
        label_list.append(label_pipeline(label))

        ## 뉴스 기사 인코딩 후 추가 
        processed_news = torch.tensor(text_pipeline(news), dtype=torch.int64)
        news_list.append(processed_news)

        ## 다음 뉴스를 읽기 위한 위치값 정보
        offsets.append(processed_news.size(0))
        #print(f'news 토큰 수 => {processed_news.size(0)}개')

    ## 배치 크기 만큼의 라벨 리스트 => 텐서화
    label_list = torch.tensor(label_list, dtype=torch.int64)

    ## 배치 크기 만큼의 길이 위치값 => 텐서화 
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    ## 배치 크기 만큼의 뉴스 기사 리스트 => 텐서화 
    news_list = torch.cat(news_list)

    return label_list.to(DEVICE), news_list.to(DEVICE), offsets.to(DEVICE)


In [11]:
## Dataset 종류 -------------------------------------------
## => Map-style Dataset
##    - 인덱스를 통한 랜덤 액세스
##    - 전처리에 유용한 데이터셋

## => Iterable-style Dataset
##    - 순차적으로 하나씩 처리 
##    - DataLoader를 통한 처리에 유용한 데이터셋
## => Map DS ==> Iterable DS 변환 함수
##    - to_map_style_dataset(Map_DS) => 결과 Iteralbe DS

In [12]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

## Map Dataset ==> Iterable Dataset 변환
trainDS = to_map_style_dataset(trainDS)
testDS  = to_map_style_dataset(testDS)

## 검증용 데이터셋위한 분할
num_train = int(len(trainDS) * 0.95)
num_valid = len(trainDS) - num_train

trainDS, validDS = random_split( trainDS, [num_train, num_valid])

print(f'trainDS : {len(trainDS)}개')
print(f'validDS : {len(validDS)}개')
print(f'testDS  : {len(testDS)}개')

trainDS : 114000개
validDS : 6000개
testDS  : 7600개


In [13]:
## => DataLoader 생성
### ===> 학습용, 검증용, 테스트용 DataSet 준비 
BATCH_SIZE = 5

### 학습용, 검증용, 테스트용 Dataset, DataLoader 준비
trainDL = DataLoader( trainDS, 
                      batch_size=BATCH_SIZE, 
                      shuffle=True, 
                      collate_fn=collate_batch )

validDL  = DataLoader( validDS, 
                       batch_size=BATCH_SIZE,  
                       shuffle=True,  
                       collate_fn=collate_batch )
                      
testDL  = DataLoader( testDS, 
                      batch_size=BATCH_SIZE,  
                      shuffle=True,  
                      collate_fn=collate_batch )

[4] 모델 클래스 정의 및 설계 <hr>

In [14]:
### ===> 모듈로딩
import torch.nn as nn
import torch.optim as optim 
from torch.optim.lr_scheduler import StepLR

In [15]:
## -------------------------------------------------------------------------
## 클래스이름 : TextModel
## 부모클래스 : Module
## 매개변수둘 : 단어사전 갯수, 임베딩 수, 분류_클래스 갯수 
## -------------------------------------------------------------------------
class TextModel(nn.Module):
    ## 모델 층 정의 메서드 --------------------------------------
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        ## 고차원 ==> 저차원 
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        ## 다중 분류 
        self.fc = nn.Linear(embed_dim, num_class)
        ## 초기 가중치 => self.메서드이름() : 같은 클래스에 존재하는 메서드 호출
        self.init_weights()

    ## 가중치 초기화 기능의 메서드 ---------------------------
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    ## 전방향 학습 메서드 -------------------------------------------
    def forward(self, text, offsets):
        ## 배치크기만큼 학습 데이터 
        embedded = self.embedding(text, offsets)
        # 다중 분류로 손실함수에서 softmax() 처리 
        return self.fc(embedded)     


[5] 학습  <hr>

==> [5-1] 학습 준비

In [34]:
## 학습 설정
EMBEDDING_DIM   = 64
LR              = 0.01
EPOCHS          = 10
STEP_SIZE		= 5

NUM_CLASS       = len(set([label for (label, text) in trainDS]))
VOCAB_SIZE      = len(vocab)

In [ ]:
## 학습 인스턴스 생성
MODEL = TextModel(  VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASS)
MODEL.to(DEVICE)

LOSS_FN   = nn.CrossEntropyLoss()
OPTIMIZER = optim.Adam(MODEL.parameters(), lr=LR)
SCHEDULER = StepLR(OPTIMIZER, STEP_SIZE, gamma=0.1)		# 현재 LR * gamma

==> [5-2]학습관련 함수들

In [ ]:
## -------------------------------------------------------------------------
## 함수기능 : 학습데이터를 사용하여 학습 진행
## 함수이름 : training
## 매개변수 : 데이터로더
## 결과반환 : 손실값, 모델성능값
## -------------------------------------------------------------------------
def training(dataloader):
    ## 학습 모드 설정
    MODEL.train()

    ## 학습 손실과 점수 저장 
    total_loss, total_acc = 0, 0
    
    for idx, (label, text, offsets) in enumerate(dataloader):

        OPTIMIZER.zero_grad()
        pre  = MODEL(text, offsets)

        loss = LOSS_FN(pre, label)
        loss.backward()
        ## gradient vanishing, gradient exploding 발생 => 방지 및 안정화 
        ## - gradient가 일정 threshold를 넘어가면 clipping
        ## - clipping: gradient의 L2norm(norm이지만 보통 L2 norm사용)으로 나눠주는 방식
        torch.nn.utils.clip_grad_norm_(MODEL.parameters(), 0.1)
        OPTIMIZER.step()

        total_loss += loss.item()
        pre_label = nn.functional.softmax(pre, dim=1)
        total_acc += (pre_label.argmax(dim=1) == label).sum().item()

        if idx==5: break
        
    return total_loss/idx+1, total_acc/idx+1



In [37]:
## -------------------------------------------------------------------------
## 함수기능 : 검증데이터를 사용하여 학습 진행
## 함수이름 : evaluate
## 매개변수 : 데이터로더
## 결과반환 : 손실값, 모델성능값
## -------------------------------------------------------------------------
def evaluate(dataloader):
    MODEL.eval()
    total_loss, total_acc = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            ## 추론 진행
            pre = MODEL(text, offsets)
            ## 손실 계산
            loss = LOSS_FN(pre, label)

            ## 손실 및 성능 평가
            total_loss += loss.item()
            total_acc += (pre.argmax(1) == label).sum().item()
            if idx==5: break
        
    return total_loss/idx+1, total_acc/idx+1

==> [5-3]학습 진행

In [38]:
## 모델 및 모델 층별 상태값 즉, 파라미터 값 저장 경로
MODEL_DIR  = '../models/'
MODEL_FILE = 'AGNEWS_MODEL.pt'

In [39]:
EPOCHS = 10  ## 임시
# 모델 저장 기준
MAX_ACC = 0.

for epoch in range(1, EPOCHS + 1):
    
    train_loss, train_acc = training(trainDL)
    valid_loss, valid_acc = evaluate(validDL)
    SCHEDULER.step()

    print("-" * 59)
    print(f'| end of epoch {epoch:3d} | train acc {train_acc:8.3f}  | valid acc {valid_acc:8.3f}')
    print("-" * 59)

    ## 모델 저장 
    if MAX_ACC < valid_acc : 
        torch.save(MODEL, MODEL_DIR+MODEL_FILE)
        MAX_ACC = valid_acc


-----------------------------------------------------------
| end of epoch   1 | train acc    3.000  | valid acc    2.600
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | train acc    2.800  | valid acc    3.000
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | train acc    2.800  | valid acc    3.000
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | train acc    3.000  | valid acc    3.400
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | train acc    4.000  | valid acc    4.400
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | train acc

[6] 모델 평가<hr>

In [40]:
print('TESTING......', end='\t')

test_loss, test_acc = evaluate(testDL)

print(f'[RESULT] Acc : {test_acc:.3f},  LOSS : {test_loss:.3f}')


TESTING......	[RESULT] Acc : 4.600,  LOSS : 2.500


[7] 예측<hr>

In [41]:
## -------------------------------------------------------------------------
## 함수기능 : 사용자데이터를 예측
## 함수이름 : predict
## 매개변수 : 모델, 텍스트, 텍스트 전처리함수
## 결과반환 : 예측값
## -------------------------------------------------------------------------
def predict(model, text, text_pipeline):
    # 검증 모드 
    model.eval()

    # 예측 진행
    with torch.no_grad():
        # 토큰화 > 정수 변환  > 텐서
        text = torch.tensor(text_pipeline(text), dtype=torch.int64)
      
        # 예측 진행
        output = model(text, torch.tensor([0]))
        # 예측 결과 반환
        return output.argmax(1).item() + 1

In [42]:
ag_news_label = {1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tec"}


### ==> 임의의 데이터 
ex_text_str_01 = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

ex_text_str_02= "As food prices continued to rise, consumer prices continued to rise in the 3% range for the second consecutive month.\
                According to the \"March Consumer Price Trend\" released by the National Statistical Office on the 2nd, the consumer price index \
                last month was 113.94 (2020 = 100), up 3.1% from the same month last year.\
                This year's consumer price growth rate increased again to 3.1% in February after recording 2.8% in January this year.\
                Prices of agricultural, livestock and fisheries products rose 11.7 percent year-on-year, up more than 11.4 percent from the previous month.\
                Among them, agricultural prices rose 20.5% year-on-year, marking the second consecutive month of increase of 20% following the previous month's 20.9%.\
                In particular, the price of apples rose 88.2 percent, which was larger than the previous month (71.0 percent), the largest increase ever since January 1980, \
                when statistics began to be compiled."


In [43]:
model = torch.load(MODEL_DIR+MODEL_FILE)

print(f"NEWS => {ag_news_label[predict(model, ex_text_str_01, text_pipeline)]}")
print(f"NEWS => {ag_news_label[predict(model, ex_text_str_02, text_pipeline)]}")

NEWS => World
NEWS => Sports
